<center><h2>Service 1: Feature Space Generation</h2></center>

<img src='https://www.projectrhea.org/rhea/images/thumb/2/22/Hyperplane.png/700px-Hyperplane.png'/>

This notebook showcases how we can utilize the ADC in order to create a feature space based on parameters such as time range, bands and buffer zone for each parcel. In addition, a daily interpolation has been applied to the data. 

<b>Import Libraries</b>

In [6]:
import h5py
import datacube
import math
import calendar
import ipywidgets
import numpy as np
import pandas as pd
import geopandas as gpd
from datacube.utils import geometry
from datacube.utils.geometry import Geometry,CRS
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import colors as mcolours
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
from datetime import datetime,timedelta
from pyproj import Proj, transform
from osgeo import ogr
from time import time
import csv
import xarray as xr
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd

<b> Set the parameters </b>

In [4]:
year = '2019'
pilot = 'cyprus'
region = 'new_area_1'
buffer = 'buffer5'

path_pilot = '/home/eouser/Desktop/jason_notebooks/shapefiles/{0}/{0}_declarations_{1}_3857.shp'.format(pilot,year)
path_val = '/home/eouser/Desktop/jason_notebooks/shapefiles/{0}/crop_declarations_validations_{0}_{1}_3857.shp'.format(pilot,year)

optical_bands = ['B02','B03','B04','B05','B06','B07','B08','B8A','B11','B12','ndvi','ndwi','psri']
sar_bands = ['vv','vh']

start_date,end_date = '2018-10-01','2019-07-01'
d_break = 8 # number of breaks between the dates interval

### Load both declarations and validations

In [9]:
dbf = gpd.GeoDataFrame.from_file(path_pilot)
print(dbf.shape)
dbf.head()

(327363, 5)


,unique_id,Decl_Code,Decl_Area,noa_id,geometry
0,1000-21/31E2-C-156~1,1,8.0,1,"POLYGON ((3716746.086 4190627.653, 3716705.329..."
1,1000-21/31E2-C-157~1,1,7.7,2,"POLYGON ((3716746.086 4190627.653, 3716691.260..."
2,1000-21/31E2-C-158~1,1,9.6,3,"POLYGON ((3716691.260 4190600.158, 3716653.480..."
3,1000-21/31E2-C-159~1,1,14.9,4,"POLYGON ((3716554.931 4190768.499, 3716572.753..."
4,1000-21/31E2-C-160~3,3,6.9,5,"POLYGON ((3716405.532 4190777.355, 3716389.080..."


In [10]:
dbf_val = gpd.GeoDataFrame.from_file(path_val)
print(dbf_val.shape)
dbf_val.head()

(49095, 11)


,unique_id,Decl_Code,Decl_Name,Val_Code,Val_Name,NMA_C,Val_Mthd,SCAT_CROP_,SCAT_TREES,TREES_FOUN,geometry
0,1012-21/61E1-E-87#1~42,42,ÎÎÎÎÎ£,42,ÎÎÎÎÎ£,1,None,NaN,0,8,"POLYGON ((3711133.127 4184532.064, 3711133.451..."
1,1012-21/61E1-E-87#2~42,42,ÎÎÎÎÎ£,42,ÎÎÎÎÎ£,1,None,NaN,0,8,"POLYGON ((3711186.855 4184558.391, 3711188.728..."
2,1021-30/12W2-D-286~42,42,ÎÎÎÎÎ£,42,ÎÎÎÎÎ£,1,None,NaN,0,8,"POLYGON ((3706845.565 4179624.019, 3706853.835..."
3,1021-30/3W1-C-251~3,3,ÎÎ¡ÎÎÎÎ¡Î,3,ÎÎ¡ÎÎÎÎ¡Î,1,None,NaN,0,0,"MULTIPOLYGON (((3703935.067 4182076.450, 37039..."
4,1023-30/31E1-F-281~3,3,ÎÎ¡ÎÎÎÎ¡Î,3,ÎÎ¡ÎÎÎÎ¡Î,1,OTS,0.0,0,0,"POLYGON ((3716441.253 4176322.531, 3716457.369..."


In [5]:
noa_to_id = {k:v for k,v in dbf[['noa_id','unique_id']].values}

In [6]:
noa_ids_to_keep = sorted(dbf[dbf.unique_id.isin(dbf_val.unique_id)]['noa_id'].values)
print(len(noa_ids_to_keep))

49095


In [11]:
config="/home/eouser/datacube.conf"
dc = datacube.Datacube(app="test",config=config)

In [10]:
shapefile_bbox = '/home/eouser/Desktop/jason_notebooks/shapefiles/{0}/{0}_bbox_{1}_3857.shp'.format(pilot,region)
driver = ogr.GetDriverByName("ESRI Shapefile")
ds = driver.Open(shapefile_bbox,0)
layer = ds.GetLayer()
xmin,xmax,ymin,ymax = layer.GetExtent()
bbox = [xmin,xmax,ymin,ymax]
bbox

[3760383.853569663, 3773903.405815693, 4157215.9410092006, 4169765.3027004637]

### Load the ids for each parcel based on a layer that has been indexed on the cube

In [11]:
product_ids = 'ids_{}_{}'.format(pilot,year)

def getIDs():
    query = {
        'product':product_ids,
        'x':(xmin,xmax),
        'y':(ymin,ymax),
        'crs':'EPSG:3857'
    }
    dc = datacube.Datacube(app="test", config=config)
    data = dc.load(**query)
    return data

In [12]:
%%time
ids = getIDs()
print('number of fields to check: {}'.format(len(np.unique(ids[buffer][0].values))-1))

number of fields to check: 10236
CPU times: user 408 ms, sys: 432 ms, total: 840 ms
Wall time: 659 ms


### Get the dates related to the data

In [13]:
def date_range(start, end, intv):

    start = datetime.strptime(start,"%Y-%m-%d")
    end = datetime.strptime(end,"%Y-%m-%d")
    diff = (end  - start ) / intv
    for i in range(intv):
        yield (start + diff * i).strftime("%Y-%m-%d")
    yield end.strftime("%Y-%m-%d")


dates_l = list(date_range(start_date,end_date,d_break))


dates_list = []
for i in range(len(dates_l)-1):
    start,end = dates_l[i],dates_l[i+1]
    start = datetime.strptime(start,"%Y-%m-%d")
    end = datetime.strptime(end,"%Y-%m-%d")
    end = end - timedelta(1)
    dates_list.append((start.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")))

dates_list

[('2018-11-01', '2018-11-30'),
 ('2018-12-01', '2018-12-30'),
 ('2018-12-31', '2019-01-29'),
 ('2019-01-30', '2019-03-01'),
 ('2019-03-02', '2019-03-31'),
 ('2019-04-01', '2019-04-30'),
 ('2019-05-01', '2019-05-30'),
 ('2019-05-31', '2019-06-30')]

### Creation of a hdf5 file to write in

In [14]:
hf = h5py.File('/home/eouser/Desktop/jason_notebooks/fs/{0}/fs_{0}_{1}_{2}_{3}_raw.h5'.format(pilot,region,year,buffer), 'a')
data = hf.create_group('data')
sar_data = hf.create_group('sar_data')
coords = hf.create_group('coords')
meta = hf.create_group('metadata')
meta.create_dataset('bands',(len(optical_bands)),'S10',[b.encode('ascii','ignore') for b in optical_bands])
meta.create_dataset('sar_bands',(len(sar_bands)),'S10',[b.encode('ascii','ignore') for b in sar_bands])

<HDF5 dataset "sar_bands": shape (2,), type "|S10">

In [14]:
def calculate_index(data, index):
    B02 = data.B02.astype('float16')
    B03 = data.B03.astype('float16')
    B04 = data.B04.astype('float16')
    B05 = data.B05.astype('float16')
    B06 = data.B06.astype('float16')
    B07 = data.B07.astype('float16')
    B08 = data.B08.astype('float16')
    B8A = data.B8A.astype('float16')
    B11 = data.B11.astype('float16')
    B12 = data.B12.astype('float16')
    if index.lower() == 'ndvi':
        return (B08 - B04) / (B08 + B04)
    if index.lower() == 'ndwi':
        return (B03 - B08) / (B08 + B03)
    if index.lower() == 'ndmi':
        return (B08 - B11) / (B08 + B11)
    if index.lower() == 'psri':
        return (B04 - B02) / B06
    if index.lower() == 'savi':
        L = 0.428;
        return ((B08 - B04) / (B08 + B04 + L)) * (1.0 + L)
    if index.lower() == 'evi':
        return 2.5 * (B08 - B04) / ((B08 + 6 * B04 - 7.5 * B02) + 1.0)
    if index.lower() == 'dvi':
        return (B08 - B04)
    if index.lower() == 'rdvi':
        return (B08 - B04) / (B08 + B04) ** 0.5
    if index.lower() == 'rvi':
        return (B08 / B04)
    if index.lower() == 'tvi':
        return (120 * (B08 - B03) - 200 * (B04 - B03)) / 2
    if index.lower() == 'tcari':
        return ((B08 - B04) - 0.2 * (B08 - B03) * (B08 / B04)) * 3
    if index.lower() == 'gi':
        return (B03 / B04)
    if index.lower() == 'vigreen':
        return (B03 - B04) / (B03 + B04)
    if index.lower() == 'varigreen':
        return (B03 - B04) / (B03 + B04 - B02)
    if index.lower() == 'gari':
        return (B08 - (B03 - (B02 - B04))) / (B08 - (B03 + (B02 - B04)))
    if index.lower() == 'gdvi':
        return (B08 - B03)
    if index.lower() == 'sipi':
        return (B08 - B02) / (B08 - B04)
    if index.lower() == 'wdrvi':
        alpha = 0.2
        return (alpha * B08 - B04) / (alpha * B08 + B04)
    if index.lower() == 'gvmi':
        return ((B08 + 0.1) - (B12 + 0.02)) / ((B08 + 0.1) + (B12 + 0.02))
    if index.lower() == 'gcvi':
        return (B08 / B03) - 1
    else:
        return None
    
def cloud_data(data, index):
    return xr.where((data.SCL>=4) & (data.SCL<=6), data[index], np.nan)



def getData_optical(bbox,timeStart,timeEnd,optical_bands,resolution = 10):
    
    product_optical= 's2_preprocessed_{}'.format(pilot)
    
    all_bands = ['B02','B03','B04','B05','B06','B07','B08','B8A','B11','B12','SCL']
    all_indices = ['ndvi','ndwi','ndmi','psri','savi','evi','dvi','rdvi','rvi','tvi','tcari','gi','vigreen',
                   'varigreen','gari','gdvi','sipi','wdrvi','gvmi','gcvi']
    
    bands = [b for b in optical_bands if b in all_bands]
    indices = [b for b in optical_bands if b in all_indices]
    
    if bbox is not None:
        xmin,xmax,ymin,ymax = bbox[0],bbox[1],bbox[2],bbox[3]
    query = {
        'time': (timeStart,timeEnd),
        'product': product_optical,
        'x':(xmin,xmax),
        'y':(ymin,ymax),
        'crs':'EPSG:3857'
    }
    dc = datacube.Datacube(app="test", config=config)
    data = dc.load(**query,measurements=all_bands,dask_chunks={})
    for i,index in enumerate(optical_bands):
        if index in indices:
            data[index] = calculate_index(data,index)
        data[index] = cloud_data(data,index)
        if i == 0:
#             to_keep = data[index].dropna(dim='time',how='all').time
            to_keep = data[index].dropna(dim='time',thresh=0.25).time
            data = data.sel(time=to_keep)

    for b in all_bands:
        if b not in bands:
            data = data.drop(b)
    return data

### Iterate over time and get pixel data for each parcel

In [16]:
flag = True

for d_n in tqdm(range(len(dates_list))):
     
        
    try:
        d_start,d_end = dates_list[d_n][0],dates_list[d_n][1]
        data_cube = getData_optical(bbox,d_start,d_end,optical_bands)
        data_cube = data_cube.load()

        ### important step: grouping of pixels per parcel based on id raster
        grouped_data = data_cube.groupby(ids[buffer][0])
    except:
        continue
    
    dates = data_cube.time.values
    dates = np.array([str(t).split('T')[0] for t in dates])
    dates_unique = sorted(set(dates))  

    
    all_keys = []
    for f in grouped_data:
        key, parcel_data = f[0],f[1]
        if key!=-1:
            coords_all = list(zip(parcel_data.x.values,parcel_data.y.values))
            parcel_data = np.array([parcel_data[b].values for b in optical_bands])

            if len(dates)!=dates_unique:
                vals = []
                for b in range(parcel_data.shape[0]):
                    df = pd.DataFrame(data=parcel_data[b],index=dates)
                    vals.append(df.groupby(df.index).mean().values)
                vals = np.array(vals)
            else:
                vals = parcel_data.copy()
            if flag:
                coords.create_dataset(str(key),data=np.array(coords_all), compression='gzip',compression_opts=9)
                data.create_dataset(str(key),data=vals.astype('float64'),compression='gzip',
                                    compression_opts=9,maxshape=(vals.shape[0],None,vals.shape[2]))
            else:
                data[str(key)].resize((data[str(key)].shape[1] + vals.shape[1]), axis=1)
                data[str(key)][:,-vals.shape[1]:,:] = vals
            all_keys.append(key)

    if flag:
        meta.create_dataset('unique_ids',(len(all_keys)),'S40',
                            [noa_to_id[i].encode('ascii','ignore') for i in np.array(sorted(all_keys))])
        meta.create_dataset('dates',(len(dates_unique)),'S10',
                            [d.encode('ascii','ignore') for d in dates_unique],maxshape=(None,))
        flag = False
    else:
        meta['dates'].resize((meta['dates'].shape[0] + len(dates_unique)), axis=0)
        meta['dates'][-len(dates_unique):] = dates_unique

100%|████████████████████████████████████████████| 8/8 [44:11<00:00, 331.42s/it]


### The same for SAR data

In [17]:
def getData_sar(bbox,timeStart,timeEnd,sar_bands,resolution = 10):
    
    product_sar= 'sentinel1_sar'
    all_sar_bands = ['vv','vh']
    
    bands = [b for b in sar_bands if b in all_sar_bands]
    if bbox is not None:
        xmin,xmax,ymin,ymax = bbox[0],bbox[1],bbox[2],bbox[3]
    query = {
        'time': (timeStart,timeEnd),
        'product': product_sar,
        'x':(xmin,xmax),
        'y':(ymin,ymax),
        'crs':'EPSG:3857'
    }
    dc = datacube.Datacube(app="test", config=config)
    data = dc.load(**query,measurements=all_sar_bands,dask_chunks={})
    for b in all_sar_bands:
        if b not in bands:
            data = data.drop(b)
    return data

flag = True

if sar_bands:
    
    for d_n in tqdm(range(len(dates_list))):
        
        try:        
            d_start,d_end = dates_list[d_n][0],dates_list[d_n][1]
            data_cube = getData_sar(bbox,d_start,d_end,sar_bands)
            data_cube = data_cube.load()
            grouped_data = data_cube.groupby(ids[buffer][0])
        except:
            continue
        sar_dates = data_cube.time.values
        sar_dates = np.array([str(t).split('T')[0] for t in sar_dates])
        dates_unique = sorted(set(sar_dates))  


        for f in grouped_data:
            key, parcel_data = f[0],f[1]
#             if key in noa_ids_to_keep:
            if key!=-1:
                parcel_data = np.array([parcel_data[b].values for b in sar_bands])

                if len(sar_dates)!=dates_unique:
                    vals = []
                    for b in range(parcel_data.shape[0]):
                        df = pd.DataFrame(data=parcel_data[b],index=sar_dates)
                        vals.append(df.groupby(df.index).mean().values)
                    vals = np.array(vals)
                else:
                    vals = parcel_data.copy()
                    
                if flag:
                    sar_data.create_dataset(str(key),data=vals.astype('float64'),compression='gzip',
                                        compression_opts=9,maxshape=(vals.shape[0],None,vals.shape[2]))
                else:
                    sar_data[str(key)].resize((sar_data[str(key)].shape[1] + vals.shape[1]), axis=1)
                    sar_data[str(key)][:,-vals.shape[1]:,:] = vals


        dates_unique = sorted(set(sar_dates))   
        if flag:
            meta.create_dataset('sar_dates',(len(dates_unique)),'S10',
                                [d.encode('ascii','ignore') for d in dates_unique],maxshape=(None,))
            flag = False
        else:
            meta['sar_dates'].resize((meta['sar_dates'].shape[0] + len(dates_unique)), axis=0)
            meta['sar_dates'][-len(dates_unique):] = dates_unique

In [19]:
hf.close()

## Filtering and Daily Linear Interpolation

In [22]:
### interpolation timestamps

def date_range(start, end, intv):

    start = datetime.strptime(start,"%Y-%m-%d")#+timedelta(days=31)
    end = datetime.strptime(end,"%Y-%m-%d")-timedelta(days=1)
    diff = (end  - start ) / intv
    for i in range(intv):
        yield (start + diff * i)
    yield end

s2_temporal_resolution = 5
s1_temporal_resolution = 6

start = datetime.strptime(start_date,"%Y-%m-%d")#+timedelta(days=31)
end = datetime.strptime(end_date,"%Y-%m-%d")-timedelta(days=1)
d_break = (end-start).days//s2_temporal_resolution
d_break_sar = (end-start).days//s1_temporal_resolution
    
dates_interp = list(date_range(start_date,end_date,d_break))
dates_interp = [d.date() for d in dates_interp]
dates_interp_sar = list(date_range(start_date,end_date,d_break_sar))
dates_interp_sar = [d.date() for d in dates_interp_sar]

In [23]:
def daily_interpolation(vals,dates_timestamp,sar=False,smoothing=False):

    df_band = pd.DataFrame(data=vals.T.copy(),columns=dates_timestamp)
    #     df_band[df_band==-9999] = np.nan
    start = datetime.strptime(start_date,"%Y-%m-%d")-timedelta(days=1)
    end = datetime.strptime(end_date,"%Y-%m-%d")+timedelta(days=1)
    df_band[start] = np.nan
    df_band[end] = np.nan
    df_band = df_band[sorted(df_band.columns)]
    df_band = df_band.resample('D',axis=1).mean().interpolate('linear',axis=1).bfill(axis=1).ffill(axis=1)
    if sar:
        df_band = df_band[dates_interp_sar]
    else:
        df_band = df_band[dates_interp]
    if smoothing:
        df_band = df_band.rolling(window=3,center=True,axis=1).median().bfill(axis=1).ffill(axis=1)
    
    return df_band.values.T

In [33]:
hf = h5py.File('/home/eouser/Desktop/jason_notebooks/fs/{0}/fs_{0}_{1}_{2}_{3}_raw.h5'.format(pilot,region,year,buffer), "r", libver='latest', swmr=True)
unique_ids = np.array([str(i.decode('UTF-8')) for i in hf['metadata']['unique_ids'][:]])
dates = np.array([str(d.decode('UTF-8')) for d in hf['metadata']['dates'][:]])
dates_timestamp = np.array([datetime.strptime(x,"%Y-%m-%d") for x in dates])
sar_dates = np.array([str(d.decode('UTF-8')) for d in hf['metadata']['sar_dates'][:]])
sar_dates_timestamp = np.array([datetime.strptime(x,"%Y-%m-%d") for x in sar_dates])
bands = np.array([str(b.decode('UTF-8')) for b in hf['metadata']['bands'][:]])
sar_bands = np.array([str(b.decode('UTF-8')) for b in hf['metadata']['sar_bands'][:]])
ids = np.array(list(hf['data'].keys()))

# ids = sorted(set(ids.astype(int)).intersection(set(noa_ids_to_keep)))
ids = np.array(ids).astype(str)
unique_ids = np.array([noa_to_id[int(i)] for i in ids])

ndvi_i = np.where(bands=='ndvi')[0][0]
lower_ndvi_thresh = 0 
sample = True # put yes if you want to extract a sample of random pixels inside the parcel
# sample_size = 0.2 # the number the sample pixels > 1 or the portion <= 1
sample_size = 10


dates_new = [d.strftime("%Y-%m-%d") for d in dates_interp]
sar_dates_new = [d.strftime("%Y-%m-%d") for d in dates_interp_sar]


hf_interp = h5py.File('/home/eouser/Desktop/jason_notebooks/fs/{0}/fs_{0}_{1}_{2}_{3}_interp.h5'.format(pilot,region,year,buffer), 'w')
data_interp = hf_interp.create_group('data')
sar_data_interp = hf_interp.create_group('sar_data')
coords_interp = hf_interp.create_group('coords')
meta_interp = hf_interp.create_group('metadata')
meta_interp.create_dataset('bands',(len(bands)),'S10',[b.encode('ascii','ignore') for b in bands])
meta_interp.create_dataset('sar_bands',(len(sar_bands)),'S10',[b.encode('ascii','ignore') for b in sar_bands])
meta_interp.create_dataset('dates',(len(dates_new)),'S10',[d.encode('ascii','ignore') for d in dates_new])
meta_interp.create_dataset('sar_dates',(len(sar_dates_new)),'S10',[d.encode('ascii','ignore') for d in sar_dates_new])
meta_interp.create_dataset('unique_ids',(len(unique_ids)),'S40',[i.encode('ascii','ignore') for i in unique_ids])


for i in tqdm(ids):
    
    vals = hf['data'][i][:]
    sar_vals = hf['sar_data'][i][:]
    coords = hf['coords'][i][:]
    
    if sample:
        vals_size = vals.shape[-1]
        if sample_size <= 1:
            sample_size = int(vals_size*sample_size)
            s = np.sort(np.random.choice(np.arange(vals_size),sample_size,replace=False))
            vals = vals[:,:,s]
            sar_vals = sar_vals[:,:,s]
            coords = coords[s,:]
        else:
            vals_size = vals.shape[-1]
            if sample_size<=vals_size:
                s = np.sort(np.random.choice(np.arange(vals_size),sample_size,replace=False))
            else:
                s = np.sort(np.random.choice(np.arange(vals_size),sample_size,replace=True))
            vals = vals[:,:,s]
            sar_vals = sar_vals[:,:,s]
            coords = coords[s,:]
    
    
    vals[vals==-9999.] = np.nan
    sar_vals[(sar_vals==0.)|(sar_vals<=-30.)] = np.nan
    
    ii = np.where(vals[ndvi_i,:,:]<=lower_ndvi_thresh) ## put nan for every band if ndvi<=ndvi_threshold
    
    vals_new = np.zeros((vals.shape[0],len(dates_new),vals.shape[-1]))
    sar_vals_new = np.zeros((sar_vals.shape[0],len(sar_dates_new),sar_vals.shape[-1]))
    for b in range(len(bands)):
        vals[b,ii[0],ii[1]] = np.nan
        vals_new[b,:,:] = daily_interpolation(vals[b,:,:],dates_timestamp,sar=False)
    for b in range(len(sar_bands)):
        sar_vals_new[b,:,:] = daily_interpolation(sar_vals[b,:,:],sar_dates_timestamp,sar=True)

    coords_interp.create_dataset(str(i),data=np.array(coords), compression='gzip',compression_opts=9)
    data_interp.create_dataset(str(i),data=vals_new.astype('float16'),compression='gzip',compression_opts=9)
    sar_data_interp.create_dataset(str(i),data=sar_vals_new.astype('float16'),compression='gzip',compression_opts=9)


100%|█████████████████████████████████████| 10236/10236 [44:24<00:00,  3.84it/s]


In [34]:
hf.close()
hf_interp.close()